# Parked: Analysis of 2023 SFMTA Parking Meter and Parking Violation Data

**Author:** Nick Gregorich
**Date:** December, 2023

This analysis was performed with 3 of my favorite tools:

1. [DuckDB](https://duckdb.org): a wonderful analytics engine and SQL interface
2. [pandas](https://pandas.pydata.org): the standard in Python data analysis
3. [Plotly Express](https://plotly.com/python/plotly-express/): my current favorite interactive plotting library

DuckDB allows us to query a [directory full of parquet files](https://duckdb.org/docs/data/parquet/overview) as if it were a table in a database with a SQL interface

Let's import these 3 packages

In [1]:
import duckdb
import pandas as pd
import plotly.express as px

We'll need to set up a few user variables:

1. `meter_data_dir`: location of the parking meter archives saved from: [DataSF](https://data.sfgov.org/Transportation/SFMTA-Parking-Meter-Detailed-Revenue-Transactions/imvp-dq3v/about_data)
    1. Acquiring this data is left as an exercise for the reader
2. `data_dir`: location of the parking violation archives saved from: [DataSF](https://data.sfgov.org/Transportation/SFMTA-Parking-Citations/ab4h-6ztd/about_data)
    1. Acquiring this data is left as an exercise for the reader
3. `large_amount_thresh`: some parking meter transactions are very large. It's unclear if these are data errors or maybe weekly or monthly passes. Either way, let's set a maximum transaction threshold to focus on what we think are probably single day transactions
4. `negative_amount_thresh`: some parking meter transactions are negative. It's unclear if these are data errors or refunds. Either way, let's focus on positive transactions
5. `sum_fines_min_thresh`: let's focus on parking violations that generated a total of over $1M in revenue over the year

In [2]:
meter_data_dir = "meter_data"
data_dir = "ticket_data"

large_amount_thresh = 30
negative_amount_thresh = 0

sum_fines_min_thresh = 1000000

The DuckDB [weekday()](https://duckdb.org/docs/sql/functions/datepart) function returns:

```Numeric weekday synonym (Sunday = 0, Saturday = 6)```

Let's create and show a *pandas DataFrame* to map the numeric weekday to the actual names 

In [3]:
day_week_dict = {0: "Sunday", 1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday"}

day_week_df = pd.DataFrame.from_dict(day_week_dict, orient="index", columns=["day"])
day_week_df

,day
0,Sunday
1,Monday
2,Tuesday
3,Wednesday
4,Thursday
5,Friday
6,Saturday


We'll want to do a histogram of the parking meter values, but it can be nicer to define the bins where we want them instead of relying on the tool to set them automatically

Let's create and show a *DataFrame* of bins with a $1 range (save for the top bin which should go to the max expected value)

**Note:** in a future version, it would probably be wise to change the very last `upper` threshold from `10000.000` to `large_amount_thresh` for consistency and clarity

In [4]:
hist_bins_df = pd.DataFrame.from_dict(
    {"lower": [0, 1.01, 2.01, 3.01, 4.01, 5.01, 6.01, 7.01, 8.01, 9.01, 10.01],
     "upper": [1.00, 2.00, 3.00, 4.00, 5.00, 6.00, 7.00, 8.00, 9.00, 10.00, 10000.00]})

hist_bins_df

,lower,upper
0,0.00,1.0
1,1.01,2.0
2,2.01,3.0
3,3.01,4.0
4,4.01,5.0
5,5.01,6.0
6,6.01,7.0
7,7.01,8.0
8,8.01,9.0
9,9.01,10.0


The violations in the dataset have a very abbreviated name

Let's create and display a *DataFrame* that maps the abbreviated name into something a little more descriptive

In [5]:
violation_dict = {
    "STR CLEAN": "Street cleaning",
    "RES/OT": "Residential parking",
    "MTR OUT DT": "Meter (out downtown)",
    "METER DTN": "Meter (downtown)",
    "NO PLATES": "No plates",
    "YEL ZONE": "Yellow zone",
    "FAIL DISPL": "Fare evasion",
    "DRIVEWAY": "Blocking driveway",
    "PK PHB OTD": "No parking (not downtown)",
    "PRK PROHIB": "No parking (downtown)",
    "DBL PARK": "Double parking",
    "BUS ZONE": "Bus zone",
    "ON SIDEWALK": "On sidewalk",
    "RED ZONE": "Red zone",
    "TRK ZONE": "Truck loading zone",
    "NO EV REG": "No registration",
    "PRK GRADE": "Parking grade",
}

violation_df = pd.DataFrame.from_dict(violation_dict, orient="index", columns=["violation_text"])
violation_df

,violation_text
STR CLEAN,Street cleaning
RES/OT,Residential parking
MTR OUT DT,Meter (out downtown)
METER DTN,Meter (downtown)
NO PLATES,No plates
YEL ZONE,Yellow zone
FAIL DISPL,Fare evasion
DRIVEWAY,Blocking driveway
PK PHB OTD,No parking (not downtown)
PRK PROHIB,No parking (downtown)


It's finally time to dive into the actual data!

First, we run a SQL `describe` on the parking meter data in order to see its schema

In the DuckDB workflow, we:

1. Define the DuckDB SQL query in a Python (f-)`string`
   1. Instead of a table name, use the [read_parquet](https://duckdb.org/docs/data/parquet/overview) function
   2. Point `read_parquet` to the directory containing the parking meter data
   3. Use a [glob](https://en.wikipedia.org/wiki/Glob_%28programming%29) to specify all the `.parquet` files in the directory
2. Run the query and use the `.to_df()` method to convert the result of the query to a pandas DataFrame
3. Display the DataFrame

**Note:** the documentation refers to `.df()` rather than `.to_df()`, so later versions may have simplified this method

In [6]:
query = f"describe select * from read_parquet('{meter_data_dir}/*.parquet');"
columns_df = duckdb.sql(query).to_df()
columns_df

,column_name,column_type,null,key,default,extra
0,transmission_datetime,VARCHAR,YES,None,None,None
1,post_id,VARCHAR,YES,None,None,None
2,street_block,VARCHAR,YES,None,None,None
3,payment_type,VARCHAR,YES,None,None,None
4,session_start_dt,VARCHAR,YES,None,None,None
5,session_end_dt,VARCHAR,YES,None,None,None
6,meter_event_type,VARCHAR,YES,None,None,None
7,gross_paid_amt,VARCHAR,YES,None,None,None


There are 8 [varchar](https://en.wikipedia.org/wiki/Varchar) columns. We can use a standard `select * from table limit 1` query to inspect a single row of the data and learn what the columns contain

**Note:** Since every column is a `varchar` *string* instead of a specialized type like `timestamp` or `double`, we need to `cast()` the varchar to the desired type in every query
 
I expect this to result in a performance hit since we are likely doing the same `cast()` repeatedly on the same data. We could `cast()` once and save the data to memory or disk, but that also may be prohibitive depending on the size of the dataset and resources available
 
The performance of `cast()` in every query was sufficient for this analysis

In [7]:
query = f"select * from read_parquet('{meter_data_dir}/*.parquet') limit 1;"
a_row_df = duckdb.sql(query).to_df()
a_row_df

,transmission_datetime,post_id,street_block,payment_type,session_start_dt,session_end_dt,meter_event_type,gross_paid_amt
0,112784135_4_01012017000836,585-02240,NORTH POINT ST 200,CASH,2017-01-01T00:08:36.000,2017-01-01T00:08:36.000,NS,0.5


Now we can see what we're dealing with in the parking meter data! Note that the DataSF link above has simple descriptions of the columns

1. `transmission_datetime = 1044369491_9_01032023131055`
    1. Using context clues from other columns, we can determine
    2. The 3rd section of the string seems to start with a date
        1. `01` month
        2. `03` day
        3. `2023` year
    3. Then there's a time
        1. `13` hour
        2. `10` minute
        3. `55` second
    4. The first section of the string may be more of a transaction number?
        1. If so, this column looks like the only unique identifier
2. `post_id = 722-28260`
    1. This is a unique identifier for the parking meter
    2. We may want to `group by` this column to compare revenue of different parking meters
3. `street_block = WEBSTER ST 2800`
    1. This is the street name and block of the parking meter
4. `payment_type = PAY BY CELL`
    1. Obviously this is the payment type of the transaction
    2. We may want to do a `group by` on this column to compare revenue from different payment methods
5. `session_start_dt = 2023-01-03T13:10:55.000`
    1. The starting datetime of the parking meter session
    2. Note that this is the same datetime we see in the first column, `transmission_datetime`
6. `session_end_dt = 2023-01-03T14:10:55.000`
    1. The ending datetime of the parking meter session
    2. This would have been nice to compare the durations of parking meter payments
    3. However, this column was never used in analysis
7. `meter_event_type = NS`
    1. Parking meter event type including:
        1. `NS = new session`
        2. `AT = additional time`
        3. `SC = status change`
    2. This column was not used in analysis
8. `gross_paid_amt = 6.75`
    1. Amount paid to the parking meter by the customer
    2. This column is the most interesting and is the basis for most of the analysis

Cool, we have a much better understanding of the parking meter data!

Let's see the datetime range of the dataset. We'll look at:

`min(session_start_dt::timestamp) as first_start_ts`, the minimum session starting datetime as the beginning of the dataset and

`max(session_start_dt::timestamp) as last_start_ts`, the maximum session starting datetime as the end of the dataset

**Note:** notice the `::timestamp` cast as mentioned above

In [8]:
query = f"""select
min(session_start_dt::timestamp) as first_start_ts,
max(session_start_dt::timestamp) as last_start_ts
from read_parquet('{meter_data_dir}/*.parquet');"""
first_last_start_df = duckdb.sql(query).to_df()
first_last_start_df

,first_start_ts,last_start_ts
0,2017-01-01 00:08:36,2023-12-16 22:38:27


The dataset started on January 1st, 2017 at 12:08:36 AM and the most recent entry as of this writing is December 16th, 2023 at 10:38:27 PM

Nearly 6 years, that's a lot of parking meter transactions!

But how many? Great question, let's find out by counting the number of rows / entries in the dataset:

In [9]:
query = f"select count(*) as count_rows from read_parquet('{meter_data_dir}/*.parquet');"
count_rows_df = duckdb.sql(query).to_df()
count_rows_df

,count_rows
0,167449468


Wow, there are 167,449,468 parking meter transactions in the 6 years of records! If we assume a constant number of transactions per year, that would be over 27 million transactions per year

One of the most important (and least glorious) aspects of working with data is *cleaning* it. We can try and learn a little bit about the data and combine it with our intuition in order to know which data is trustworthy and which is not. This can be things like duplicate data, missing data, or outliers

We might expect all parking meter transactions to be positive since we generally don't get paid *by* meters to park, so let's see if there are any negative transaction amounts and if so, how many

**Note:** notice the `gross_paid_amt` column is cast to `double`

In [10]:
query = f"""select count(gross_paid_amt) as count_rows
from read_parquet('{meter_data_dir}/*.parquet')
where gross_paid_amt::double < {negative_amount_thresh};"""
count_negative_amount_df = duckdb.sql(query).df()
count_negative_amount_df

,count_rows
0,8


There are only 8 (out of 167 million) records that have negative dollar amounts. That's a very small percentage, so it is unlikely to meaningfully affect our analysis (we could look at the actual amounts to estimate their affect), but we'll still filter them out

We might expect parking meter transaction amounts to have a reasonable upper limit. We chose a threshold of $30 in the user variables section above, which feels like it should cover the range *normal* parking meter transactions

Let's see how many transactions are greater than our upper threshold

In [11]:
query = f"""select count(gross_paid_amt) as count_rows
from read_parquet('{meter_data_dir}/*.parquet')
where gross_paid_amt::double > {large_amount_thresh};"""
count_large_amount_df = duckdb.sql(query).df()
count_large_amount_df

,count_rows
0,145363


There are 145,363 transactions over $30 in the dataset, significantly more than the negative transactions! It would be nice to understand these better, but for this analysis we'll just filter them out

Let's focus on transactions in 2023 and see how many there were. We'll:

1. Cast the session start `varchar` to a `date`
2. Take the `year()` of the `date`
3. Filter where `year()` is 2023
4. Filter where transactions are between 0 and 30 (inclusive)
5. `group by` the year so we can `count()` the number of payments
    1. In a future version, I would put `year(session_start_dt::date)` in the `where` so I could drop the `group by`

In [12]:
query = f"""
select year(session_start_dt::date) as year, count(gross_paid_amt) as count_payments
from read_parquet('{meter_data_dir}/*.parquet') as data_t
where year = 2023
and gross_paid_amt::double between {negative_amount_thresh} and {large_amount_thresh}
group by year;
"""
payment_count_2023_df = duckdb.sql(query).to_df()
payment_count_2023_df

,year,count_payments
0,2023,17478748


There were 17,478,748 parking meter transactions in 2023. Although the year wasn't quite over, that is significantly lower than the 27 million transactions per year we estimated above

How much revenue did these 17 million transactions generate?

1. Start with the previous query
2. Instead of counting the number of transaction amounts
    1. Sum the amount paid (cast to `double`)

In [13]:
query = f"""
select year(session_start_dt::date) as year, sum(gross_paid_amt::double) as sum_amount_paid
from read_parquet('{meter_data_dir}/*.parquet') as data_t
where year = 2023
and gross_paid_amt::double between {negative_amount_thresh} and {large_amount_thresh}
group by year;
"""
amount_sum_2023_df = duckdb.sql(query).to_df()
amount_sum_2023_df

,year,sum_amount_paid
0,2023,4.727701e+07


Wow, over $47M was collected in parking meter transactions! That's a solid business!

Above, we noted that the payment method is present in the table. Let's take a look by doing a `group by` on `payment_type` 

In [14]:
query = f"""
select payment_type, sum(gross_paid_amt::double) as sum_amount_paid
from read_parquet('{meter_data_dir}/*.parquet') as data_t
where year(session_start_dt::date) = 2023
and gross_paid_amt::double between {negative_amount_thresh} and {large_amount_thresh}
group by payment_type
order by payment_type
"""
amount_type_2023_df = duckdb.sql(query).to_df()
amount_type_2023_df

,payment_type,sum_amount_paid
0,CASH,3.631406e+06
1,CREDIT CARD,2.822013e+07
2,PAY BY CELL,1.520821e+07
3,SMART CARD,2.172594e+05


It looks like there are 4 payment methods in the dataset:

1. Cash: &#36;3.6M
2. Credit card: &#36;28M
3. Pay by cell phone: &#36;15M
4. Smart card: &#36;217k

Credit cards were the most common payment method at &#36;28M followed by pay by cell phone around half that at &#36;15M. Cash was an order of magnitude lower at &#36;3.6M and a legacy payment method called smart card was another order of magnitude lower at &#36;217k

If we get these numbers as a fraction of the total, we can build a nice pie or donut chart for our infographic!

Here we do some manipulation of the *pandas DataFrame* saved above in order to generate the fractions of the whole (aka percentages). In hindsight this was a little bit of a strange decision since we did all of our previous data manipulation in *DuckDB* / *SQL*

1. Create a Python dictionary with keys of payment method and the aggregate sum of the payment amounts
    1. For simplicity, the legacy smart cards are lumped in with credit cards
2. Create a *pandas DataFrame* from the dictionary, rename and reindex
3. Create a new `fraction` column in the DataFrame
4. Set the `fraction` column to the amount paid per transaction method over the total of all transaction amounts 

In [15]:
amount_type_2023_dict = {
    "CASH": amount_type_2023_df[amount_type_2023_df.payment_type == "CASH"].iloc[0].sum_amount_paid,
    "CREDIT CARD": amount_type_2023_df[amount_type_2023_df.payment_type == "CREDIT CARD"].iloc[0].sum_amount_paid + \
                   amount_type_2023_df[amount_type_2023_df.payment_type == "SMART CARD"].iloc[0].sum_amount_paid,
    "PAY BY CELL": amount_type_2023_df[amount_type_2023_df.payment_type == "PAY BY CELL"].iloc[0].sum_amount_paid,
}
amount_type_2023_df = pd.DataFrame.from_dict(amount_type_2023_dict, orient="index")
amount_type_2023_df = amount_type_2023_df.rename(columns={0: "sum_amount_paid"})
amount_type_2023_df["payment_type"] = amount_type_2023_df.index
amount_type_2023_df = amount_type_2023_df.reset_index()
amount_type_2023_df["fraction"] = amount_type_2023_df.sum_amount_paid / amount_type_2023_df.sum_amount_paid.sum()
amount_type_2023_df

,index,sum_amount_paid,payment_type,fraction
0,CASH,3.631406e+06,CASH,0.076811
1,CREDIT CARD,2.843739e+07,CREDIT CARD,0.601506
2,PAY BY CELL,1.520821e+07,PAY BY CELL,0.321683


Nice, we can see credit card was 60% of transactions by dollar amount, pay by cell phone was 32% by amount, and cash was just 7.7% by amount

Let's see a pie chart of this data using *Plotly Express*

In [16]:
fig = px.pie(amount_type_2023_df, values="sum_amount_paid")
fig.update_traces(textposition='inside', textinfo='none')
fig.show()

Nice, the first chart for our infographic! Thanks Plotly!

It would be interesting to know some details about the parking meter transaction amounts. Are the bulk of the transactions $1 quick stops or are they higher dollar, presumably longer duration events?

This will be our most ambitious query yet:

1. The `from` will be the same `read_parquet()` *table* we've been using
2. `cross join` to the `hist_bins_df` DataFrame we created above
    1. `cross join` is a rare join method but possibly the simplest: it returns all the possible pairing between both tables
    2. `hist_bins_df` is the DataFrame of histogram ranges we created above
        1. It contains a lower and upper limit (inclusive) for each bin we want in our histogram
3. `select` the `lower` and `upper` limits from `hist_bins_df`
4. `group by` the same `lower` and `upper` limits so we can do an aggregate count of the number of parking meter payments
5. Use the same filters as before
    1. Year is 2023
    2. Transaction amount is between 0 and 30 (inclusive)
6. `order by` the upper limit
7. After the *DuckDB SQL* query
    1. Use pandas to add a `fraction` of each bin of the total number of transactions
    2. Use pandas to add a `proportion` of each fraction relative to the max fraction
        1. In our infographic, we may want to make relatively sized objects. The `proportion` column will help there

**Note:** in a future version, I would order `lower` before `upper`. We also don't need to do an aggregate count of the `gross_paid_amt` `cast()` to `double`

In [17]:
query = f"""
select hist_bins_df.upper, hist_bins_df.lower, count(gross_paid_amt::double) as count_rows
from read_parquet('{meter_data_dir}/*.parquet')
cross join hist_bins_df
where year(session_start_dt::date) = 2023
and gross_paid_amt::double between {negative_amount_thresh} and {large_amount_thresh}
and gross_paid_amt::double between hist_bins_df.lower and hist_bins_df.upper
group by hist_bins_df.lower, hist_bins_df.upper
order by hist_bins_df.upper
"""
amount_2023_hist_df = duckdb.sql(query).to_df()
amount_2023_hist_df["fraction"] = amount_2023_hist_df.count_rows / amount_2023_hist_df.count_rows.sum()
amount_2023_hist_df["proportion"] = amount_2023_hist_df.fraction / amount_2023_hist_df.fraction.max()
amount_2023_hist_df

,upper,lower,count_rows,fraction,proportion
0,1.0,0.00,6877770,0.393493,1.000000
1,2.0,1.01,3291349,0.188306,0.478549
2,3.0,2.01,1993588,0.114058,0.289860
3,4.0,3.01,1494789,0.085520,0.217336
4,5.0,4.01,1414455,0.080924,0.205656
5,6.0,5.01,574052,0.032843,0.083465
6,7.0,6.01,490803,0.028080,0.071361
7,8.0,7.01,312368,0.017871,0.045417
8,9.0,8.01,222548,0.012732,0.032358
9,10.0,9.01,162231,0.009282,0.023588


That's a nice table of results!

1. In the first 2 columns, we see the range of each histogram bin
2. In the `count_rows` column, we see the number of transactions that fall into each bin
3. In the `fraction` column, we see each bin's / row's fraction of the total number of transactions
4. In the `proportion` column, we see each row's `fraction` normalized by the largest `fraction`

The greatest number of transactions was in the &#36;0 - &#36;1 range, with over 6.8 million transactions which is over 39% of the total number of transactions. Note that the proportion is 1.0 here

The number of transactions in the &#36;1 - &#36;2 range is half of the &#36;0 - &#36;1. Numbers decrease as you may expect until the &#36;10 - &#36;10,000 range. This makes sense, the bin is very large

**Note:** as mentioned previously, it was confusing to have the top bin go to &#36;10,000 when the query is filtering to &#36;30

Another thing we could look at is the amount of parking meter revenue by day of the week. We can:

1. Cast the parking session start datetime to `date`, then use the `weekday()` function to extract the day of the week
2. `group by` the day of the week so we can aggregate the sum of the amount paid (cast to `double`)
3. Calculate the fraction and proportion in pandas, as we did above

In [18]:
query = f"""
select weekday(session_start_dt::date) as day_week, sum(gross_paid_amt::double) as sum_amount_paid
from read_parquet('{meter_data_dir}/*.parquet')
where year(session_start_dt::date) = 2023
and gross_paid_amt::double between {negative_amount_thresh} and {large_amount_thresh}
group by day_week;
"""
amount_day_hist_df = duckdb.sql(query).to_df()
amount_day_hist_df = amount_day_hist_df.merge(day_week_df, left_on="day_week", right_index=True)
amount_day_hist_df["fraction"] = amount_day_hist_df.sum_amount_paid / amount_day_hist_df.sum_amount_paid.sum()
amount_day_hist_df["proportion"] = amount_day_hist_df.fraction / amount_day_hist_df.fraction.max()
amount_day_hist_df

,day_week,sum_amount_paid,day,fraction,proportion
0,0,1124208.46,Sunday,0.023779,0.134735
1,1,7106789.91,Monday,0.150322,0.851741
2,2,7535308.73,Tuesday,0.159386,0.903099
3,3,7722644.45,Wednesday,0.163349,0.925551
4,4,7481758.58,Thursday,0.158254,0.896681
5,5,7962459.22,Friday,0.168421,0.954292
6,6,8343835.96,Saturday,0.176488,1.000000


Interesting results, we see fairly similar revenue (and therefore fraction) per day -- except on Sunday! This makes sense, many parking meters do not require payment on Sunday. It's unclear if some parking meters do in fact collect on Sunday or if people mistakenly pay on a day that is free. Sunday still has a respectable revenue of over $1M

Next we could look at is the amount of parking meter revenue by hour of the day. We can take the last query and substitute the `weekday()` function with `hour()`

In [19]:
query = f"""
select hour(session_start_dt::timestamp) as hour, sum(gross_paid_amt::double) as sum_amount_paid
from read_parquet('{meter_data_dir}/*.parquet')
where year(session_start_dt::date) = 2023
and gross_paid_amt::double between {negative_amount_thresh} and {large_amount_thresh}
group by hour;
"""
amount_hour_hist_df = duckdb.sql(query).to_df()
amount_hour_hist_df["fraction"] = amount_hour_hist_df.sum_amount_paid / amount_hour_hist_df.sum_amount_paid.sum()
amount_hour_hist_df["proportion"] = amount_hour_hist_df.fraction / amount_hour_hist_df.fraction.max()
amount_hour_hist_df

,hour,sum_amount_paid,fraction,proportion
0,0,114.40,0.000002,0.000018
1,1,93.65,0.000002,0.000015
2,2,52.05,0.000001,0.000008
3,3,79.45,0.000002,0.000013
4,4,33047.07,0.000699,0.005238
5,5,109934.97,0.002325,0.017425
6,6,267409.40,0.005656,0.042384
7,7,829156.10,0.017538,0.131421
8,8,1552226.78,0.032833,0.246028
9,9,5806223.29,0.122813,0.920286


This looks like it will make a nice visualization!

That's interesting, the highest revenue by hour is at noon: folks getting lunch and / or running errands at lunch? There are relative maxima at 9 AM, 11 AM, and 1 PM too

Maybe we could find the highest grossing parking meters to get an idea of where the hotspots are around town

1. `group by` the `street_block` column
2. Aggregate the sum of the amount paid

In [20]:
query = f"""
select street_block, sum(gross_paid_amt::double) as sum_amount_paid
from read_parquet('{meter_data_dir}/*.parquet') as data_t
where gross_paid_amt::double between {negative_amount_thresh} and {large_amount_thresh}
and year(session_start_dt::date) = 2023
group by street_block
order by sum_amount_paid desc;
"""
amount_meter_2023_df = duckdb.sql(query).to_df()
amount_meter_2023_df["fraction"] = amount_meter_2023_df.sum_amount_paid / amount_meter_2023_df.sum_amount_paid.sum()
amount_meter_2023_df["proportion"] = amount_meter_2023_df.fraction / amount_meter_2023_df.fraction.max()
amount_meter_2023_df

,street_block,sum_amount_paid,fraction,proportion
0,UNION ST 2100,266266.58,5.632053e-03,1.000000e+00
1,CHESTNUT ST 2200,238154.79,5.037434e-03,8.944224e-01
2,POST ST 1700,237352.73,5.020469e-03,8.914101e-01
3,CHESTNUT ST 2100,225117.34,4.761667e-03,8.454585e-01
4,CHANNEL ST 200,217342.39,4.597211e-03,8.162586e-01
...,...,...,...,...
1739,HARRISON ST 100,2.00,4.230386e-08,7.511269e-06
1740,POLK ST 100,1.50,3.172790e-08,5.633452e-06
1741,EMBARCADERO SOUTH 500,1.18,2.495928e-08,4.431649e-06
1742,EMBARCADERO NORTH 900,1.00,2.115193e-08,3.755635e-06


Nice, we have a list of 1,743 parking meters, the total revenue in 2023 per meter, the fraction of the total revenue per meter, and the proportion normalized to the maximum revenue per meter

We could use a plot to visualize the revenue per parking meter to get an idea of the distribution: are all meters created equal?

In [21]:
fig = px.line(amount_meter_2023_df, y="sum_amount_paid",
              title=f"Amount paid per SF parking meter in 2023")
fig.update_layout(xaxis_title='Meter number')
fig.update_yaxes(title_text='Total ($)')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

To answer our own question, no, not all parking meters are created equal. A small fraction of parking meters generate on the order of a quarter million dollars a year, the vast majority seem to collect in the &#36;10,000 to &#36;100,000 range, and a sizeable tail collect less than &#36;10,000 a year

Let's see the max revenue from a parking meter:

In [22]:
amount_meter_2023_df.sum_amount_paid.max()

266266.58000000054

&#36;266,266 from a single parking meter, wow!

Let's see the median revenue collected from a meter:

In [23]:
amount_meter_2023_df.sum_amount_paid.median()

16501.43500000002

The median is &#36;16.5k, a far cry from the maximum value!

OK, let's take a look at a different dataset from DataSF: parking citations

We can go a little bit quicker this time. First, we'll inspect the columns of the dataset with a `describe`

In [24]:
query = f"describe select * from read_parquet('{data_dir}/*.parquet');"
columns_df = duckdb.sql(query).to_df()
columns_df

,column_name,column_type,null,key,default,extra
0,citation_number,VARCHAR,YES,None,None,None
1,citation_issued_datetime,VARCHAR,YES,None,None,None
2,violation,VARCHAR,YES,None,None,None
3,violation_desc,VARCHAR,YES,None,None,None
4,citation_location,VARCHAR,YES,None,None,None
5,vehicle_plate_state,VARCHAR,YES,None,None,None
6,vehicle_plate,VARCHAR,YES,None,None,None
7,fine_amount,VARCHAR,YES,None,None,None
8,date_added,VARCHAR,YES,None,None,None
9,the_geom,VARCHAR,YES,None,None,None


This dataset has 10 columns, also all `varchar` strings. Let's query a single row so we can get familiar with the data:

In [25]:
query = f"select * from read_parquet('{data_dir}/*.parquet') limit 1;"
a_row_df = duckdb.sql(query).to_df()
a_row_df

,citation_number,citation_issued_datetime,violation,violation_desc,citation_location,vehicle_plate_state,vehicle_plate,fine_amount,date_added,the_geom
0,756689080,2008-12-16T12:17:00.000,T202,PRK METER,1720 CHURCH ST,CA,5NQR857,50,2008-12-16T00:00:00.000,None


Let's look at this one in detail:

1. `citation_number = 848824174`
    1. This one's pretty straight forward
    2. We expect it to be a unique identifier
2. `citation_issued_datetime = 2014-11-26T08:10:00.000`
    1. Again, probably straight forward
3. `violation = TRC7.2.22`
    1. Violation code as found [here](https://www.sfmta.com/sites/default/files/reports-and-documents/2022/05/fy_2023_fees_and_fines_effective_7.1.22.pdf)
4. `violation_desc = STR CLEAN`
    1. A slightly more detailed but still fairly short description of the violation
    2. A longer description is available [here](https://www.sfmta.com/sites/default/files/reports-and-documents/2022/05/fy_2023_fees_and_fines_effective_7.1.22.pdf)
5. `citation_location = 799 SHRADER ST`
    1. Straight forward
    2. Might be interesting to look at violations within a radius to find citation hot spots
        1. This is not included in this analysis
6. `vehicle_state_plate = CA`
    1. Straight forward
7. `vehicle_plate = 4SWC745`
    1. Straight forward
    2. We could aggregate the sum by `vehicle_plate` to find the offenders with the highest fines per year
8. `fine_amount = 66`
    1. Straight forward
    2. Probably the basis for most of the analysis of this dataset
9. `date_added = 2014-11-26T00:00:00.000`
    1. Date added to dataset (according to DataSF)
    2. We probably won't use this data
10. `the_geom = None`
    1. Geojson point of citation location (according to DataSF)
    2. I believe this is rarely populated, so we can ignore it

**Note:** this data was not originally checked for duplicates and unfortunately there are duplicates. To de-duplicate the data, we could do a big `group by` and select the lowest (assumed to be original) `date_added`. Something like:

```sql
select
    citation_number,
    citation_issued_datetime,
    violation,
    violation_desc,
    citation_location,
    vehicle_state_plate,
    vehicle_plate,
    fine_amount,
    min(date_added) as original_date_added,
    the_geom
 from
    read_parquet('{data_dir}/*.parquet')
 group by
    citation_number,
    citation_issued_datetime,
    violation,
    violation_desc,
    citation_location,
    vehicle_state_plate,
    vehicle_plate,
    fine_amount,
    the_geom
```

Similar to the parking meter data, we can get the sum of all the fines in 2023:

In [26]:
query = f"""select sum(fine_amount::double) as sum_fines
from read_parquet('{data_dir}/*.parquet')
where
year(citation_issued_datetime::date) = 2023;"""
fines_sum_2023_df = duckdb.sql(query).to_df()
fines_sum_2023_df

,sum_fines
0,99944796.0


Wow, over $99M in parking violations in 2023 alone! I wonder how many violations that was

In [27]:
query = f"""select count(fine_amount) as count_rows
from read_parquet('{data_dir}/*.parquet')
where
year(citation_issued_datetime::date) = 2023;"""
fines_count_2023_df = duckdb.sql(query).to_df()
fines_count_2023_df

,count_rows
0,1014481


There were 1,014,481 citations. A quick spot check is &#36;99M / 1M citations = &#36;99 per citation. That seems about right

In San Francisco, many streets have a posted street cleaning time window where parking is not allowed that can be once a week or even every day downtown. It would probably be interesting to see a breakdown of total fine amounts for street cleaning versus all other violations. We can:

1. Combine 2 queries with a `union`
    1. A query summing the fines for street cleaning violations in 2023
    2. A query summing the fines for violations that are not street cleaning in 2023
2. Calculate the fraction in pandas  

In [28]:
query = f"""select 'Street cleaning' as violation_desc, sum(fine_amount::double) as sum_fines
from read_parquet('{data_dir}/*.parquet')
where
year(citation_issued_datetime::date) = 2023
and violation_desc = 'STR CLEAN'
union
select 'Other' as violation_desc, sum(fine_amount::double) as sum_fines
from read_parquet('{data_dir}/*.parquet')
where
year(citation_issued_datetime::date) = 2023
and violation_desc <> 'STR CLEAN';"""
fines_clean_other_2023_df = duckdb.sql(query).to_df()
fines_clean_other_2023_df[
    "fraction"] = fines_clean_other_2023_df.sum_fines / fines_clean_other_2023_df.sum_fines.sum()
fines_clean_other_2023_df

,violation_desc,sum_fines,fraction
0,Street cleaning,42169326.0,0.421926
1,Other,57775470.0,0.578074


Wow, over 42% of fines by violation are from street cleaning for a sum of over &#36;42M. That's a huge revenue stream just from street cleaning violations!

Let's look at the violations per hour of the day, but let's also add in weekend vs weekday to see if there is any difference. We can:

1. `cast()` the citation datetime to weekday to get day of the week
2. Use a `case when` to populate a `varchar` with `weekday` or `weekend` accordingly
3. Pull out the hour of the day
4. `group by` the day of the week (type) and hour
5. Aggregate the sum of fines in each group

In [29]:
query = f"""select
case when weekday(citation_issued_datetime::timestamp) in (0, 6) then 'weekend'
else 'weekday' end as type,
hour(citation_issued_datetime::timestamp) as hour,
sum(fine_amount::double) sum_fines
from read_parquet('{data_dir}/*.parquet')
where
year(citation_issued_datetime::date) = 2023
group by type, hour
order by type, hour;"""
fine_day_week_hour_2023_df = duckdb.sql(query).to_df()
fine_day_week_hour_2023_df[
    "fraction"] = fine_day_week_hour_2023_df.sum_fines / fine_day_week_hour_2023_df.sum_fines.sum()
fine_day_week_hour_2023_df[
    "proportion"] = fine_day_week_hour_2023_df.fraction / fine_day_week_hour_2023_df.fraction.max()
fine_day_week_hour_2023_df

,type,hour,sum_fines,fraction,proportion
0,weekday,0,2798589.0,0.028001,0.187804
1,weekday,1,438252.0,0.004385,0.029410
2,weekday,2,1558177.0,0.015590,0.104564
3,weekday,3,190295.0,0.001904,0.012770
4,weekday,4,626620.0,0.006270,0.042050
5,weekday,5,123620.0,0.001237,0.008296
6,weekday,6,4381211.0,0.043836,0.294008
7,weekday,7,5940298.0,0.059436,0.398633
8,weekday,8,8293514.0,0.082981,0.556549
9,weekday,9,11958339.0,0.119649,0.802483


Interesting, just like the peak parking meter revenue, the peak parking citation revenue is also at 12 PM on a weekday!

There are 48 rows of data here, so it would likely be much easier to see it visually

Since we know it's a large portion of total fines, we could also break out by hour and street cleaning violations versus others for weekdays

In [30]:
query = f"""select
case when weekday(citation_issued_datetime::timestamp) in (0, 6) then 'weekend'
else 'weekday' end as type,
case when violation_desc = 'STR CLEAN' then 'Street cleaning'
else 'Other' end as cleaning_or_other,
hour(citation_issued_datetime::timestamp) as hour,
sum(fine_amount::double) sum_fines
from read_parquet('{data_dir}/*.parquet')
where
year(citation_issued_datetime::date) = 2023
and weekday(citation_issued_datetime::timestamp) not in (0, 6)
group by type, hour, cleaning_or_other
order by type, hour, cleaning_or_other desc;
"""
clean_other_weekday_hour_2023_df = duckdb.sql(query).to_df()
clean_other_weekday_hour_2023_df[
    "fraction"] = clean_other_weekday_hour_2023_df.sum_fines / clean_other_weekday_hour_2023_df.sum_fines.sum()
clean_other_weekday_hour_2023_df[
    "proportion"] = clean_other_weekday_hour_2023_df.fraction / clean_other_weekday_hour_2023_df.fraction.max()
clean_other_weekday_hour_2023_df

,type,cleaning_or_other,hour,sum_fines,fraction,proportion
0,weekday,Street cleaning,0,2273016.0,2.522806e-02,0.265620
1,weekday,Other,0,525573.0,5.833302e-03,0.061417
2,weekday,Street cleaning,1,389456.0,4.322548e-03,0.045511
3,weekday,Other,1,48796.0,5.415838e-04,0.005702
4,weekday,Street cleaning,2,1485342.0,1.648572e-02,0.173574
5,weekday,Other,2,72835.0,8.083911e-04,0.008511
6,weekday,Street cleaning,3,135288.0,1.501553e-03,0.015809
7,weekday,Other,3,55007.0,6.105193e-04,0.006428
8,weekday,Street cleaning,4,596460.0,6.620072e-03,0.069701
9,weekday,Other,4,30160.0,3.347440e-04,0.003524


Just glancing at these results, we can see the peak is street cleaning violations at 9 AM on a weekday with a sum of &#36;8.5M!

Now let's do the same for weekends

In [31]:
query = f"""select
case when weekday(citation_issued_datetime::timestamp) in (0, 6) then 'weekend'
else 'weekday' end as type,
case when violation_desc = 'STR CLEAN' then 'Street cleaning'
else 'Other' end as cleaning_or_other,
hour(citation_issued_datetime::timestamp) as hour,
sum(fine_amount::double) sum_fines
from read_parquet('{data_dir}/*.parquet')
where
year(citation_issued_datetime::date) = 2023
and weekday(citation_issued_datetime::timestamp) in (0, 6)
group by type, hour, cleaning_or_other
order by type, hour, cleaning_or_other desc;
"""
clean_other_weekend_hour_2023_df = duckdb.sql(query).to_df()
clean_other_weekend_hour_2023_df[
    "fraction"] = clean_other_weekend_hour_2023_df.sum_fines / clean_other_weekend_hour_2023_df.sum_fines.sum()
clean_other_weekend_hour_2023_df[
    "proportion"] = clean_other_weekend_hour_2023_df.fraction / clean_other_weekend_hour_2023_df.fraction.max()
clean_other_weekend_hour_2023_df

,type,cleaning_or_other,hour,sum_fines,fraction,proportion
0,weekend,Street cleaning,0,708.0,0.000072,0.000718
1,weekend,Other,0,264860.0,0.026900,0.268706
2,weekend,Street cleaning,1,36696.0,0.003727,0.037229
3,weekend,Other,1,68948.0,0.007003,0.069949
4,weekend,Street cleaning,2,761304.0,0.077320,0.772358
5,weekend,Other,2,65472.0,0.006650,0.066423
6,weekend,Street cleaning,3,47376.0,0.004812,0.048064
7,weekend,Other,3,41085.0,0.004173,0.041682
8,weekend,Street cleaning,4,355791.0,0.036135,0.360957
9,weekend,Other,4,23376.0,0.002374,0.023715


Here we can spot a peak on the weekend at 11 AM, and it's for anything but street cleaning with a sum just shy of $1M

Maybe we've been too focused on street cleaning citations, let's look at all citations by total fines

1. `group by` `violation` and `violation_desc`
2. Aggregate the sum of the fines per group
3. Filter results where sum is over our user variable threshold of &#36;1M
4. Join the resulting DataFrame with a longer description defined above
5. Calculate the fraction and proportion of each violation

In [32]:
query = f"""select
violation,
violation_desc,
sum(fine_amount::double) sum_fines
from read_parquet('{data_dir}/*.parquet')
where
year(citation_issued_datetime::date) = 2023
group by violation, violation_desc
having sum_fines > {sum_fines_min_thresh}
order by sum_fines desc;"""
fine_amounts_2023_df = duckdb.sql(query).to_df()
fine_amounts_2023_df = fine_amounts_2023_df.merge(violation_df, left_on="violation_desc", right_index=True)
fine_amounts_2023_df[
    "fraction"] = fine_amounts_2023_df.sum_fines / fine_amounts_2023_df.sum_fines.sum()
fine_amounts_2023_df[
    "proportion"] = fine_amounts_2023_df.fraction / fine_amounts_2023_df.fraction.max()
fine_amounts_2023_df

,violation,violation_desc,sum_fines,violation_text,fraction,proportion
0,TRC7.2.22,STR CLEAN,42169326.0,Street cleaning,0.476720,1.000000
1,TRC7.2.20,RES/OT,9191161.0,Residential parking,0.103905,0.217958
2,TRC7.2.23B,MTR OUT DT,8302948.0,Meter (out downtown),0.093864,0.196895
3,TRC7.2.23A,METER DTN,3777134.0,Meter (downtown),0.042700,0.089571
4,V5200,NO PLATES,3347266.0,No plates,0.037841,0.079377
5,TRC7.2.26,YEL ZONE,2815452.0,Yellow zone,0.031828,0.066765
6,TRC7.2.101A,FAIL DISPL,2451750.0,Fare evasion,0.027717,0.058141
7,V22500E,DRIVEWAY,2373100.0,Blocking driveway,0.026828,0.056276
8,TRC7.2.41,PK PHB OTD,2323836.0,No parking (not downtown),0.026271,0.055107
9,TRC7.2.40,PRK PROHIB,2313792.0,No parking (downtown),0.026157,0.054869


As we know, the top violation is street cleaning. This is followed by residential parking violations, expired parking meters which are broken out by downtown and not, no plates, and yellow zone. The next violation isn't technically a parking citation at all, it's a fare evasion on SFMTA MUNI public transportation

Next we can `group by` license plate to see the vehicles with the highest total fines

In [33]:
query = f"""select
vehicle_plate_state,
vehicle_plate,
count(fine_amount) as count_fines,
sum(fine_amount::double) as sum_fines
from read_parquet('{data_dir}/*.parquet')
where
year(citation_issued_datetime::date) = 2023
and vehicle_plate <> ''
and fine_amount::double > 0
group by vehicle_plate_state, vehicle_plate
order by sum_fines desc;"""
fine_plate_2023_df = duckdb.sql(query).to_df()
fine_plate_2023_df

,vehicle_plate_state,vehicle_plate,count_fines,sum_fines
0,CA,70426W1,171,18180.0
1,CA,EP10498,155,17136.0
2,CA,6AWB601,177,15558.0
3,CA,75566B2,117,14494.0
4,CA,11932J2,133,14063.0
...,...,...,...,...
455559,CA,8VAG274,1,23.0
455560,CA,6XMD261,1,23.0
455561,CA,8UZE304,1,23.0
455562,CA,7LSN343,1,23.0


It looks like there were citations for over 455k unique license plates

Let's see the max total fines for a single vehicle:

In [34]:
fine_plate_2023_df.sum_fines.max()

18180.0

Wow, over &#36;18k in fines for a single vehicle. That's more than a fair number of vehicles are worth!

What about the median?

In [35]:
fine_plate_2023_df.sum_fines.median()

108.0

The median fines per plate in the dataset is &#36;108. That is almost 1/200th of the maximum fines

Finally, we can view every 100th item in the citations per license plate. This was used in creating a plot showing the distribution of fines per license plate

In [36]:
fine_plate_2023_df[::100]

,vehicle_plate_state,vehicle_plate,count_fines,sum_fines
0,CA,70426W1,171,18180.0
100,CA,41830M3,55,6161.0
200,CA,9CEM456,53,5083.0
300,CA,9FEY698,44,4385.0
400,CA,36430D1,33,3928.0
...,...,...,...,...
455100,CA,7NES221,1,28.0
455200,CA,K063B0,1,23.0
455300,CA,9EEV992,1,23.0
455400,CA,MOVEBOI,1,23.0


Well, this is all the data used to generate the *Parked 2023* infographic

Nick 